Mount Google Drive (optional)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# # os.chdir("/content/drive/MyDrive/....")  # file path
# print(os.getcwd())

# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part






In [2]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [4]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_values = np.array(input_data.values)
features = np.array(input_values[:,0:9])
labels = np.array(input_values[:,9])

## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [5]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

In [6]:
ans_features = []
ans_thresholds = []

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [7]:
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """
  entropy_value = 0.
  cnt_false = 0
  cnt_true = 0
  total = data['diabetes_mellitus'].size
  # No subtree exsist
  if(total == 0):
    return 0
  # Count sunbtree
  for i in data['diabetes_mellitus']:
    if(i == 0):
      cnt_false += 1
    elif(i == 1):
      cnt_true += 1
  # Probability
  pro_true = cnt_true / total
  pro_false = cnt_false / total
  if(cnt_false == 0 or cnt_true == 0):  # Pure Tree
    entropy_value = 0    
  else:
    entropy_value = (-pro_true) * math.log2(pro_true) - (pro_false) * math.log2(pro_false)
  
  return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.9871377743721863


In [8]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """
  ig = 0
  entro_before = entropy(data)
  
  # Subtree entropy
  left_sub = data[mask.isin([True])]
  right_sub = data[mask.isin([False])]
  entro_left = entropy(left_sub)
  entro_right = entropy(right_sub)
 
  # Coefficient
  left_size = left_sub.shape[0]
  right_size = right_sub.shape[0]
  total_size = data.shape[0]
  
  ig = entro_before - (left_size / total_size * entro_left +  right_size / total_size * entro_right)

  return ig

# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)

ans_informationGain =  0.08345988684807149


## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [9]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """
  best_ig = 0
  best_threshold = 0
  best_feature = ''
  # Go through all features
  for feature in data:
    if(feature == 'diabetes_mellitus'):
      continue
    sorted_data = data.sort_values(by = [feature])
    flag = 1
    last_val = 0
    
    # Find threshold and compute ig
    for now_val in sorted_data[feature]:
      if(flag):
          last_val = now_val
          flag = 0
          continue
      threshold = (now_val + last_val) / 2
      last_val = now_val
      
      # Split Data into two branch
      mask = np.zeros((data.shape[0], 1), dtype=bool)
      idx = 0
      for i in range(data.shape[0]):
        idx +=1
        if(data[feature][i] <= threshold):
          mask[i] = True
        elif(data[feature][i] > threshold):
          mask[i] = False
      
      # Update ig
      df_mask = pd.DataFrame(mask, columns=['mask'])
      now_ig = information_gain(data, df_mask['mask'])
      if(now_ig > best_ig):
        best_ig = now_ig
        best_threshold = threshold
        best_feature = feature
  
      
  return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)

ans_ig =  0.3522950515812332
ans_value =  235.5
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [10]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  left = pd.DataFrame(data[data[feature] <= threshold]).reset_index(drop=True)
  right = pd.DataFrame(data[data[feature] > threshold]).reset_index(drop=True)
  
  return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)

ans_left =  10


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [11]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """
  # check the condition of current depth and the remaining number of samples
  if depth < max_depth and data.shape[0] > min_samples_split :
    # call find_best_split() to find the best combination
    ig, threshold, feature = find_best_split(data)
    # check the value of information gain is greater than 0 or not 
    if ig > 0 :
      # update the depth
      depth += 1
      # call make_partition() to split the data into two parts
      left_sub, right_sub = make_partition(data, feature, threshold)
      # If there is no data split to the left tree OR no data split to the left tree
      if left_sub.shape[0] == 0 or right_sub.shape[0] == 0:
        # return the label of the majority
        label = data[['diabetes_mellitus']].mode(axis='index').iloc[0, 0]
        return label
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}
        ans_features.append(feature)
        ans_thresholds.append(threshold)

        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left_sub, max_depth, min_samples_split, depth)
        right_subtree = build_tree(right_sub, max_depth, min_samples_split, depth)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      label = data[['diabetes_mellitus']].mode(axis='index').iloc[0, 0]
      return label
  else:
    # return the label of the majority
    label = data[['diabetes_mellitus']].mode(axis='index').iloc[0, 0]
    return label

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [12]:
decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

In [13]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features

['glucose_apache', 'heart_rate_apache']

In [14]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds

[235.5, 143.5]

## Step6 : Save answers

In [15]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [16]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [17]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [5]:
from sklearn.metrics import confusion_matrix
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  equation = list(tree.keys())[0] 
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  predict_data = pd.DataFrame(data).reset_index(drop=True)
  y_prediction = []
  # [Note] You can call the function classify_data() to predict the label of each instance
  for i in range(data.shape[0]):
    y_prediction.append(classify_data(predict_data.iloc[i], tree))

  return y_prediction


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  score = f1_score(y_true, y_pred)
  
  return score

y_true = [1,0,0,1,1,0,1,0,0]
y_pred = [0,0,1,1,1,0,1,1,0]
print(calculate_score(y_true, y_pred))
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print(tn, fp, fn, tp)

0.6666666666666665
3 2 1 3


In [19]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

ans_f1score =  0.6666666666666666


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [20]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.9871377743721863, 0.08345988684807149, 0.3522950515812332, 235.5, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', 235.5, 143.5, 0.6666666666666666]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [21]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')

You can split *advanced_data* into training set and validaiton set

In [22]:
num_train = 7000
num_validation = 1379

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(advanced_data.shape)
print(training_data.shape)
print(validation_data.shape)

(8379, 25)
(7000, 25)
(1379, 25)


## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [23]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


    ```
    max_depth = 
    depth = 0
    min_samples_split = 
    
    # total number of trees in a random forest
    n_trees = 

    # number of features to train a decision tree
    n_features = 

    # the ratio to select the number of instances
    sample_size = 
    n_samples = int(training_data.shape[0] * sample_size)
    ```




In [24]:
# Define the attributes
max_depth = 4
depth = 0
min_samples_split = 35 

n_trees = 3 
n_features = 8  
sample_size = 0.1
n_samples = 500 

In [25]:
def build_forest(data, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """

  forest = []
  cnt = 0
  training_round = 0
  # must reuse function build_tree()
  for i in range(n_trees):
    shuffle_data = data.sample(n=n_samples, axis=0, random_state = random.randint(1, 2**32 -1)) # sample in row(example)
    sample_feature = shuffle_data.iloc[:, :24].sample(n=n_features, axis=1, random_state = random.randint(1, 2**32 -1)) # sample in column(feature)
    label_feature = shuffle_data.iloc[:, 24:25] # diabetes_mellitus
    n_data = pd.concat([sample_feature, label_feature], axis=1).reset_index(drop=True)

    tree = build_tree(n_data, max_depth, min_samples_split, depth)
    if(tree == 0 or tree == 1):
      i -= 1
      continue
    # y_pred = make_prediction(tree, x_validation)
    # f1 = f1_score(y_validation, y_pred)
    training_round += 1
    print("Now Training Rounds: ", training_round)
    forest.append(tree)
    
  return forest

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [26]:
def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """
  score = []
  y_pred = []
  y_prediction = np.zeros(data.shape[0])
  tree_num = len(forest)

  if(data.shape[0] == 840): # For testing data
    for tree in forest:
      y_pred.append(make_prediction(tree, data))
  else:                     # For validation data
    cnt = 1
    for tree in forest:
      y_pred.append(make_prediction(tree, data))
      print(f"Tree{cnt:2d} F1_score:",calculate_score(y_validation, y_pred[-1]))
      cnt += 1
    
  df_y = pd.DataFrame(y_pred[0])
  for i in range(1, tree_num):
      df_y = pd.concat([df_y, pd.DataFrame(y_pred[i])], axis=1)
  
  for i in range(df_y.shape[0]):
    if(df_y.loc[i].sum() > tree_num / 2):
      y_prediction[i] = 1
    else:
      y_prediction[i] = 0

  return y_prediction

In [27]:
forest = build_forest(training_data, n_trees, n_features, n_samples)

Now Training Rounds:  1
Now Training Rounds:  2
Now Training Rounds:  3


In [28]:
y_pred = make_prediction_forest(forest, pd.DataFrame(x_validation))
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

Tree 1 F1_score: 0.6504559270516718
Tree 2 F1_score: 0.5519332827899923
Tree 3 F1_score: 0.5363457760314342
ans_f1score =  0.6214592274678111


In [29]:
y_pred_test = make_prediction_forest(forest, x_test)

## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [30]:
advanced = []
for i in range(len(y_pred_test)):
  advanced.append(int(y_pred_test[i]))

In [31]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)